In [2]:
from selenium import webdriver

In [3]:
driver = webdriver.Firefox()
driver.get("http://www.naver.com/")

# 다나와 사이트 크롤링

In [4]:
driver.get("http://prod.danawa.com/list/?cate=112758&searchOption=btnAllOptUse=true/searchMaker%5B%5D=702/innerSearchKeyword=")

In [5]:
import pandas as pd
import pickle

In [6]:
post= driver.find_elements_by_css_selector("#productListArea")[0].find_elements_by_css_selector("div.main_prodlist.main_prodlist_list")[0]

In [32]:
post_elements = post.find_elements_by_css_selector("div.prod_main_info")

In [33]:
a = post_elements[0].text.strip()
a

"LG전자 PC그램 14Z950-GT55L\n인텔 / 코어i5-5세대 / 브로드웰 / i5-5200U (2.2GHz) / 35.56cm(14인치) / 1920x1080 / LED백라이트 / 광시야각 / 8GB / DDR3L / M.2 / 500GB / ODD 옵션(선택) / HD 5500 / 시스템 메모리 공유 / 100Mbps 유선랜 / 802.11 n/ac 무선랜 / WiDi / 블루투스 4.0 / HDMI / 웹캠 / USB 2.0 / USB 3.0 / 멀티 리더기 / 블록 키보드 / 리튬 폴리머 / 탈착 불가능 / 윈도우8.1 / 13.4mm / 980g / 구성변경상품 / 색상: 화이트\n관련기사\n리뷰안테크, LG그램 노트북용 SSD '850X' 시리즈 선봬\n등록월 2015.03\n관심상품\n2몰\n1,539,000원\nSSD 500GB"

In [42]:
import re
pattern = re.compile("[SSDDRLmTAM23.]{3,5} / [0-9]{1,3}[G,T]{1}B")
pattern2 = re.compile("[0-9]?[.]?[0-9]{1,3}[ K]{0,2}?g")
pattern2.findall(a)

['980g']

In [64]:
df = pd.DataFrame(columns=["CPU" , "CPU2", "CPU3", "Ram" , "Memory_type" ,"Memory_size", "Weight" ,"Size", "Price"])

In [65]:
import re
def crawling():
    for post_element in post_elements:
        spec = post_element.find_element_by_css_selector("div.spec_list").text.strip()
        price = post_element.find_element_by_css_selector("div.prod_pricelist").text.strip()
        cpu_pattern = re.compile("코어i[0-9]{1}")
        cpu2_pattern = re.compile("[0-9]{1}세대")
        cpu3_pattern = re.compile("[0-9 ]{4}[A-Z]{1,2}")
        ram_pattern = re.compile(" [0-9]{1,2}GB")
        memory_pattern = re.compile("[SSDDRLmTAM234.]{3,5} / [0-9]{1,3}[G,T]{1}B")
        weight_pattern = re.compile("[0-9]?[.]?[0-9]{1,3}[ K]{0,2}?g")
        size_pattern = re.compile("[]?{1,2}[0-9]{1,2}[.]?[0-9 ]{1,3}인치")
        price_pattern = re.compile("[0-9]?[,]?[0-9]{2,3}[,][0-9]{3}원")

        cpu = cpu_pattern.findall(spec)[0]
        cpu2 = cpu2_pattern.findall(spec)[0]
        cpu3 = cpu3_pattern.findall(spec)[0]
        ram = ram_pattern.findall(spec)[0]
        memory = memory_pattern.findall(spec)
        if memory:
            memory_type = memory_pattern.findall(spec)[0].split("/")[0]
        else:
            memory = "HDD"
        
        memory_size = memory_pattern.findall(spec)[0].split("/")[1]

        weight = weight_pattern.findall(spec)
        if weight:
            weight = weight_pattern.findall(spec)[0]
        else:
            weight = "None"
        size = size_pattern.findall(spec)
        if size:
            size = size_pattern.findall(spec)[0]
        else:
            size = "None"
        
        try:
            price = price_pattern.findall(price)[0]

            data = {"CPU" : cpu , "CPU2": cpu2 , "CPU3":cpu3 , "Ram" : ram , "Memory_type" : memory_type ,"Memory_size": memory_size, "Weight" : weight , "Size":size , "Price" : price}
            df.loc[len(df)] = data

        except:
            pass

    return df

    
    

In [57]:
# 자바스크립트 페이지가져오기 (movePage)

In [58]:
url = driver.find_elements_by_css_selector("div.number_wrap")[0].find_elements_by_css_selector("a")[0].get_attribute("href")

In [59]:
driver.get(url)

In [60]:
import time
for i in range(1,18):
    time.sleep(3)
    driver.execute_script("movePage({i})".format(i = i))
    time.sleep(3)
    post= driver.find_elements_by_css_selector("#productListArea")[0].find_elements_by_css_selector("div.main_prodlist.main_prodlist_list")[0]
    time.sleep(3.6)
    post_elements = post.find_elements_by_css_selector("div.prod_main_info")
    time.sleep(3)
    crawling()

# 주의:
Memory_type에 보조기억장치(SSD,M.2..)가 아닌  DDR3L이 적혀있을 경우 HDD임을 뜻한다.
이 점은 나중에 전처리를 할 예정!! 
### SSD, M.2 -> SSD로 통일
### DDR3L => SSD가 없어 Ram타입이나온것으로, HDD로 전처리 할 예정

In [61]:
df

,CPU,CPU2,CPU3,Ram,Memory_type,Memory_size,Weight,Size,Price
0,코어i5,6세대,6200U,8GB,M.2,256GB,980g,15.6인치,"1,194,000원"
1,코어i5,6세대,6200U,8GB,DDR3L,1TB,1.89Kg,15.6인치,"875,000원"
2,코어i3,6세대,6100U,8GB,DDR3L,500GB,1.79Kg,15.6인치,"633,000원"
3,코어i5,6세대,6200U,8GB,M.2,256GB,980g,14인치,"1,014,000원"
4,코어i7,6세대,6500U,8GB,M.2,256GB,980g,15.6인치,"1,351,000원"
5,코어i3,6세대,6100U,4GB,DDR3L,500GB,1.79Kg,15.6인치,"542,000원"
6,코어i5,6세대,6200U,8GB,M.2,256GB,980g,14인치,"1,229,000원"
7,코어i5,5세대,5200U,8GB,M.2,256GB,980g,13.3인치,"882,000원"
8,코어i3,6세대,6100U,8GB,M.2,256GB,980g,14인치,"895,000원"
9,코어i5,6세대,6200U,4GB,DDR3L,1TB,2.11Kg,15.6인치,"758,000원"


In [62]:
#df.to_pickle("DanawaLg.pkl")

In [4]:
import pickle
pickle.load(open("DanawaLg.pkl","rb"))

,CPU,CPU2,CPU3,Ram,Memory_type,Memory_size,Weight,Size,Price
0,코어i5,6세대,6200U,8GB,M.2,256GB,980g,15.6인치,"1,194,000원"
1,코어i5,6세대,6200U,8GB,DDR3L,1TB,1.89Kg,15.6인치,"875,000원"
2,코어i3,6세대,6100U,8GB,DDR3L,500GB,1.79Kg,15.6인치,"633,000원"
3,코어i5,6세대,6200U,8GB,M.2,256GB,980g,14인치,"1,014,000원"
4,코어i7,6세대,6500U,8GB,M.2,256GB,980g,15.6인치,"1,351,000원"
5,코어i3,6세대,6100U,4GB,DDR3L,500GB,1.79Kg,15.6인치,"542,000원"
6,코어i5,6세대,6200U,8GB,M.2,256GB,980g,14인치,"1,229,000원"
7,코어i5,5세대,5200U,8GB,M.2,256GB,980g,13.3인치,"882,000원"
8,코어i3,6세대,6100U,8GB,M.2,256GB,980g,14인치,"895,000원"
9,코어i5,6세대,6200U,4GB,DDR3L,1TB,2.11Kg,15.6인치,"758,000원"


In [3]:
#lg_df.to_csv("DanawaLg.csv")